In [5]:
import os
import numpy as np
import pandas as pd
import modin.pandas as mpd
import random

from tqdm import tqdm
import torch
from torch import Tensor

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from torch_geometric.loader import LinkNeighborLoader

from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

from torch_geometric.nn import GATConv, to_hetero
import torch.nn.functional as F

In [6]:
savefd = "model/nodes4_tag/"
os.makedirs(savefd, exist_ok=True)

In [24]:
playlist_song = mpd.DataFrame(pd.read_parquet("data/gnn_playlists2songs.parquet"))
song_artist = mpd.DataFrame(pd.read_parquet("data/gnn_songs2artists.parquet"))
playlist_tag = mpd.DataFrame(pd.read_parquet("data/gnn_playlists2tags.parquet"))
song_class = mpd.DataFrame(pd.read_parquet("data/gnn_songs2classes.parquet"))
song_style = mpd.DataFrame(pd.read_parquet("data/gnn_songs2styles.parquet"))
song_genre = mpd.DataFrame(pd.read_parquet("data/gnn_songs2genres.parquet"))


In [25]:
song_class['class'] = song_class['class'].str[-4:].values.astype(int)
song_style['style'] = song_style['style'].str[-4:].values.astype(int)
song_genre['genre'] = song_genre['genre'].str[-4:].values.astype(int)

In [26]:
song_genre.head(50)

,song_id,genre
0,419103,185
1,607460,93
2,735213,32
3,978750,3
4,1675601,180
5,1761949,1
6,1865665,34
7,3016958,12
8,3113648,1
9,3161051,36


In [4]:
unique_playlist_id = np.unique(playlist_song['playlist_id'].values)
unique_playlist_id = mpd.DataFrame(data={
    'playlist_id': unique_playlist_id,
    'mappedID': pd.RangeIndex(len(unique_playlist_id)),
})
unique_song_id = np.unique(playlist_song['song_id'].values)
unique_song_id = mpd.DataFrame(data={
    'song_id': unique_song_id,
    'mappedID': pd.RangeIndex(len(unique_song_id)),
})
unique_artist = np.unique(song_artist['artist_id'].values)
unique_artist = mpd.DataFrame(data={
    'artist': unique_artist,
    'mappedID': pd.RangeIndex(len(unique_artist)),
})
unique_tag = np.unique(playlist_tag['tag_id'].values)
unique_tag = mpd.DataFrame(data={
    'tag': unique_tag,
    'mappedID': pd.RangeIndex(len(unique_tag)),
})

In [5]:
def make_edge(edge_df, u0, u1):
    edge_df.drop_duplicates(inplace=True)
    ekey0, ekey1 = list(edge_df.columns)
    ukey0 = u0.columns[0]
    ukey1 = u1.columns[0]
    temp0 = mpd.merge(edge_df[ekey0], u0, left_on=ekey0, right_on=ukey0, how='left')
    temp0 = torch.from_numpy(temp0['mappedID'].values)
    temp1 = mpd.merge(edge_df[ekey1], u1, left_on=ekey1, right_on=ukey1, how='left')
    temp1 = torch.from_numpy(temp1['mappedID'].values)
    return torch.stack([temp0, temp1], dim=0)

edge_playlist_song = make_edge(playlist_song, unique_playlist_id, unique_song_id)
edge_song_artist = make_edge(song_artist, unique_song_id, unique_artist)
edge_playlist_tag = make_edge(playlist_tag, unique_playlist_id, unique_tag)

In [6]:
data = HeteroData()

data["playlist"].node_id = torch.arange(len(unique_playlist_id))
data["song"].node_id = torch.arange(len(unique_song_id))
data["artist"].node_id = torch.arange(len(unique_artist))
data["tag"].node_id = torch.arange(len(unique_tag))

data["playlist", "playlist2song", "song"].edge_index = edge_playlist_song
data["song", "song2artist", "artist"].edge_index = edge_song_artist
data["playlist", "playlist2tag", "tag"].edge_index = edge_playlist_tag

data = T.ToUndirected()(data)

In [7]:
from torch_geometric.nn import GATConv, to_hetero
import torch.nn.functional as F

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, heads=8):
        super().__init__()
        self.conv1 = GATConv(hidden_channels, hidden_channels, heads, dropout=0.6, add_self_loops=False)
        self.conv2 = GATConv(hidden_channels * heads, hidden_channels, heads=1, dropout=0.6, add_self_loops=False)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

    
class Classifier(torch.nn.Module):
    def forward(self, x_playlist: Tensor, x_song: Tensor, edge_label_index: Tensor) -> Tensor:
        edge_feat_playlist = x_playlist[edge_label_index[0]]
        edge_feat_song = x_song[edge_label_index[1]]
        return (edge_feat_playlist * edge_feat_song).sum(dim=-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.playlist_emb = torch.nn.Embedding(data["playlist"].num_nodes, hidden_channels)
        self.song_emb = torch.nn.Embedding(data["song"].num_nodes, hidden_channels)
        self.tag_emb = torch.nn.Embedding(data["tag"].num_nodes, hidden_channels)
        self.artist_emb = torch.nn.Embedding(data["artist"].num_nodes, hidden_channels)
        self.gnn = GNN(hidden_channels)
        self.gnn = to_hetero(self.gnn, metadata=data.metadata())
        self.classifier = Classifier()

    def forward(self, data: HeteroData) -> Tensor:
        x_dict = {
          "playlist": self.playlist_emb(data["playlist"].node_id),
          "song": self.song_emb(data["song"].node_id),
          "artist": self.artist_emb(data["artist"].node_id),
          "tag": self.tag_emb(data["tag"].node_id),
        } 

        x_dict = self.gnn(x_dict, data.edge_index_dict)
        pred = self.classifier(
            x_dict["playlist"],
            x_dict["song"],
            data["playlist", "playlist2song", "song"].edge_label_index,
        )

        return pred

        
model = Model(hidden_channels=64)
batch_size = 1024 * 4
epochs = 1000

In [8]:
from torch_geometric.loader import LinkNeighborLoader

transform = T.RandomLinkSplit(
    num_val=0,
    num_test=0,
    disjoint_train_ratio=0,
    neg_sampling_ratio=2.0,
    add_negative_train_samples=False,
    edge_types=("playlist", "playlist2song", "song"),
    rev_edge_types=("song", "rev_playlist2song", "playlist"), 
)

train_data, val_data, test_data = transform(data)

In [9]:
train_loader = LinkNeighborLoader(
    data=train_data,
    num_neighbors=[32, 16],
    neg_sampling_ratio=2.0,
    edge_label_index=(("playlist", "playlist2song", "song"), train_data["playlist", "playlist2song", "song"].edge_label_index),
    edge_label=train_data["playlist", "playlist2song", "song"].edge_label,
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

pbar = tqdm(total=epochs*len(train_loader))
for epoch in range(epochs):
    total_loss = total_examples = 0
    model.train()
    for sampled_data in train_loader:
        optimizer.zero_grad()

        sampled_data.to(device)
        pred = model(sampled_data)

        ground_truth = sampled_data["playlist", "playlist2song", "song"].edge_label
        loss = F.binary_cross_entropy_with_logits(pred, ground_truth)

        loss.backward()
        optimizer.step()
        pbar.update(1)
    
    filename = f"{savefd}/epoch_{str(epoch).zfill(4)}.pth"
    torch.save(model, filename)
    pbar.set_description(f"Epoch: {epoch}")
    # print(filename, " saved")

Device: 'cuda'


  0%|          | 133/466000 [01:01<59:30:25,  2.17it/s]